# Libraries:

In [1]:
string_1 = "INTENTION"
string_2 = "EXECUTION"

# string_1 = "NOITNETIN"
# string_2 = "NOITUCEXE"

M = len(string_1) # + 1
N = len(string_2) # + 1

In [2]:
a_star_array = [[0 for cell in range(M + 1)] for cell in range(N + 1)]

In [3]:
class operations():
    def __init__(self, insert, delete, substitute):
        self.insert = insert
        self.delete = delete
        self.substitute = substitute

In [4]:
operations_array = [[operations(0, 0, 0) for cell in range(M + 1)] for cell in range(N + 1)]

# Initialization:

`D(i, 0)`

In [5]:
for i in range(1, N + 1):
    a_star_array[i][0] = i

`D(0, j)`

In [6]:
for j in range(1, M + 1):
    a_star_array[0][j] = j

# Recurrence Relation

In [7]:
# fill up the matrix
for i in range(1, N + 1):
    for j in range(1, M + 1):
        operations_array[i][j].delete = a_star_array[i-1][j] + 1
        operations_array[i][j].insert = a_star_array[i][j-1] + 1

        if string_1[j-1] != string_2[i-1]:
            operations_array[i][j].substitute = a_star_array[i-1][j-1] + 2
        elif string_1[j-1] == string_2[i-1]:
            operations_array[i][j].substitute = a_star_array[i-1][j-1] + 0
        else:
            print("Error in Substitute Case") 
        a_star_array[i][j] = min(
                                operations_array[i][j].delete, 
                                operations_array[i][j].insert, 
                                operations_array[i][j].substitute
                             )

In [8]:
import pandas as pd
a_star_array_df = pd.DataFrame(a_star_array)
a_star_array_df

,0,1,2,3,4,5,6,7,8,9
0,0,1,2,3,4,5,6,7,8,9
1,1,2,3,4,3,4,5,6,7,8
2,2,3,4,5,4,5,6,7,8,9
3,3,4,5,6,5,6,7,8,9,10
4,4,5,6,7,6,7,8,9,10,11
5,5,6,7,8,7,8,9,10,11,12
6,6,7,8,7,8,9,8,9,10,11
7,7,6,7,8,9,10,9,8,9,10
8,8,7,8,9,10,11,10,9,8,9
9,9,8,7,8,9,10,11,10,9,8


In [ ]:
import heapq

def a_star_edit_distance(s1, s2):
    m, n = len(s1), len(s2)
    # heap stores: (f_score, g_score, i, j)
    # Heuristic: simple length difference
    start_h = abs(m - n)
    open_set = [(start_h, 0, 0, 0)]
    
    came_from = {}
    g_score = {(0, 0): 0}

    while open_set:
        f, cost, i, j = heapq.heappop(open_set)

        # Goal check
        if i == m and j == n:
            # We found the path!
            # Return the detailed path info AND the cost
            return reconstruct_path(came_from, i, j, s1, s2), cost

        # Define possible moves
        moves = []
        if i < m: 
            moves.append((1, 0, 1, "Delete"))       # Delete from s1
        if j < n: 
            moves.append((0, 1, 1, "Insert"))       # Insert from s2
        if i < m and j < n:
            sub_cost = 0 if s1[i] == s2[j] else 2
            # CHANGED: Used "Keep" instead of "Match"
            moves.append((1, 1, sub_cost, "Keep" if sub_cost == 0 else "Substitute"))

        for di, dj, move_cost, op in moves:
            ni, nj = i + di, j + dj
            new_g = cost + move_cost
            
            if (ni, nj) not in g_score or new_g < g_score[(ni, nj)]:
                g_score[(ni, nj)] = new_g
                h = abs((m - ni) - (n - nj))
                f = new_g + h
                heapq.heappush(open_set, (f, new_g, ni, nj))
                came_from[(ni, nj)] = (i, j, op)

def reconstruct_path(came_from, i, j, s1, s2):
    path_steps = []
    
    # Lists to build the visual alignment block
    align_top = [] # Source string
    align_mid = [] # Operation codes (K, S, D, I)
    align_bot = [] # Target string

    curr = (i, j)
    while curr in came_from:
        prev_i, prev_j, op = came_from[curr]
        
        step_str = ""
        
        # LOGIC: Build the strings backwards
        # CHANGED: Check for "Keep" instead of "Match"
        if op == "Keep" or op == "Substitute":
            char_s1 = s1[prev_i]
            char_s2 = s2[prev_j]
            step_str = f"{op} ({prev_i},{prev_j}) [{char_s1}] -> ({curr[0]},{curr[1]}) [{char_s2}]"
            
            align_top.append(char_s1)
            align_bot.append(char_s2)
            # CHANGED: Use "K" for Keep visualization
            align_mid.append("K" if op == "Keep" else "S")
            
        elif op == "Delete":
            char_s1 = s1[prev_i]
            step_str = f"{op} ({prev_i},{prev_j}) [{char_s1}] -> ({curr[0]},{curr[1]}) [-]"
            
            align_top.append(char_s1)
            align_bot.append("-")    
            align_mid.append("D")
            
        elif op == "Insert":
            char_s2 = s2[prev_j]
            step_str = f"{op} ({prev_i},{prev_j}) [-] -> ({curr[0]},{curr[1]}) [{char_s2}]"
            
            align_top.append("-")
            align_bot.append(char_s2)
            align_mid.append("I")

        path_steps.append(step_str)
        curr = (prev_i, prev_j)
    
    # Reverse everything because we backtraced from the end
    return path_steps[::-1], "".join(align_top[::-1]), "".join(align_mid[::-1]), "".join(align_bot[::-1])

# --- Execution ---
source_str = "INTENTION"
target_str = "EXECUTION"

# 1. Get the tuple (path_details, cost)
path_details, total_cost = a_star_edit_distance(source_str, target_str)

# 2. Unpack the path_details tuple into the 4 components
steps, top_row, mid_row, bot_row = path_details

# --- Output ---
print(f"Source: {source_str}")
print(f"Target: {target_str}")
print("-----")
print(f"Distance: {total_cost}")
print("Alignment:")
print(f"{top_row}")
print(f"{mid_row}") 
print(f"{bot_row}")

print("\nDetailed Steps:")
for step in steps:
    print(step)